# Model Development

-----------

## Install necessary libraries

``` pip install tensorflow keras matplotlib scikit-learn ```

------

## Organize image data

In [ ]:
import os
import shutil
import random

# Define paths
original_dataset_dir = "path/to/original_images"
base_dir = "path/to/split_data"
os.makedirs(base_dir, exist_ok=True)

"""
# Directories for train, validation, test
splits = ['train', 'validation', 'test']
for split in splits:
    for jump_type in os.listdir(original_dataset_dir):
        os.makedirs(os.path.join(base_dir, split, jump_type), exist_ok=True)
"""

# Split data
split_ratios = {'train': 0.7, 'validation': 0.15, 'test': 0.15}
for jump_type in os.listdir(original_dataset_dir):
    images = os.listdir(os.path.join(original_dataset_dir, jump_type))
    random.shuffle(images)
    
    train_end = int(len(images) * split_ratios['train'])
    val_end = train_end + int(len(images) * split_ratios['validation'])
    
    for i, image in enumerate(images):
        if i < train_end:
            split = 'train'
        elif i < val_end:
            split = 'validation'
        else:
            split = 'test'
        src = os.path.join(original_dataset_dir, jump_type, image)
        dest = os.path.join(base_dir, split, jump_type, image)
        shutil.copy(src, dest)

-----

## Model Work

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Paths (update these to your dataset paths)
train_dir = "path/to/train"
val_dir = "path/to/validation"
test_dir = "path/to/test"

### Data preparation

In [ ]:
# Step 1: Data Preparation
IMG_SIZE = 224  # Standard size for ResNet50 input
BATCH_SIZE = 32

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

### Load pre-trained model

In [ ]:
# Step 2: Load Pre-trained Model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Add a dense layer
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Final output layer

model = Model(inputs=base_model.input, outputs=predictions)

### Compile the model

In [ ]:
# Step 3: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [ ]:
# Step 4: Train the Model
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

### Evaluate the model

In [ ]:
# Step 5: Evaluate the Model
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Classification Report
y_true = test_generator.classes
y_pred = np.argmax(model.predict(test_generator), axis=-1)
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 7))
plt.imshow(cm, cmap='Blues')
plt.colorbar()
plt.title("Confusion Matrix")
plt.show()